# ANÁLISE LOTOFÁCIL  (CONCURSOS E GANHADORES)

 - Quantidade de concursos por ano e mês
 - Quantidade de ganhadores por ano e mês  
 - Análise de série temporal  
 

**Observação:** `Será necessário alterando para o diretório raíz do projeto para poder fazer imports locais`

In [1]:
%cd ..

c:\Users\Vedovi\lotofacil\lotofacil


### Importações

In [2]:
# Ignorar avisos do Arima
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning  
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', ValueWarning)

# Importa a função para gerar a base de dados
from processamento.indice_resultado import dados_indice

# Importando pacote para análise de série temporal
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

# Criar gráfico
import plotly.graph_objs as go

# Para mostrar o gráfico no notebook 
import plotly.offline as py
py.init_notebook_mode(connected=True)

# Criar gráfico
import matplotlib.pyplot as plt
%matplotlib inline

# Para formatar datas
from datetime import datetime

### Funções

In [3]:

def grafico_linha_plt(x, y, title, xlabel, ylabel):
    """
    Função para criar um gráfico de linhas utilizando o Matplotlib.
    
    Parâmetros:
    x      -> Valores do eixo (x)
    y      -> Valores do eixo (y)
    title  -> Título do gráfico
    xlabel -> Rótulo do eixo (x)
    ylabel -> Rótulo do eixo (y)
    
    """
    
    # Define as dimenssões do gráfico
    plt.figure(figsize=(15, 5))
    
    # Define o título, rótulo x, rótulo y
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    # Cria o gráfico
    plt.plot(x, y, color='purple', marker='o', linestyle='dashed', linewidth=1.5, markersize=4)
    # Formata a linha de grade do gráfico
    plt.grid(linestyle='--', which='major', color='grey', alpha=.25)
    # Visualiza o gráfico
    plt.show()
    
    
def grafico_linha_go(x, y, title, xlabel, ylabel):
    """
    Função para criar um gráfico de linhas utilizando o Plotly.
    
    Parâmetros:
    x      -> Valores do eixo (x)
    y      -> Valores do eixo (y)
    title  -> Título do gráfico
    xlabel -> Rótulo do eixo (x)
    ylabel -> Rótulo do eixo (y)
    
    """
    
    # Criando o gráfico
    trace = go.Scatter(x=x, y=y, mode='markers+lines')

    # Armazenando gráfico em uma lista
    grafico = [trace]

    # Criando Layout
    layout = go.Layout(title=title, xaxis={'title': xlabel}, yaxis={'title': ylabel})

    # Criando figura que será exibida
    fig = go.Figure(data=grafico, layout=layout)

    # Visualiza o gráfico
    py.iplot(fig)

### Definição e análise dos dados

In [4]:
# Criando a base de dados
base = dados_indice()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Vedovi\\lotofacil\\lotofacil\\lotofacil\\base\\resultados.csv'

In [ ]:
# Mostra as informações da base de dados
base.info()

In [ ]:
# Converte para o tipo data
base['Data Sorteio'] = base['Data Sorteio'].apply(lambda _: datetime.strptime(_, '%d/%m/%Y'))

In [ ]:
# Visualizando os tipos dos dados
base.dtypes

In [ ]:
# Visualiza as 5 primeiras linhas
base.head()

In [ ]:
# Quantidade de concursos por ano
concursos_ano = base.groupby('Ano')['Concurso'].count()
concursos_ano

In [ ]:
# Quantidade de ganhadores por ano
ganhadores = base.groupby('Ano')['Ganhou'].sum()
ganhadores

In [ ]:
# Definindo os dados para analisar a quantidade de concursos por ano
x_concursos = []
y_concursos = []

for k, v in dict(concursos_ano).items():
    x_concursos.append(k)
    y_concursos.append(v)

In [ ]:
# Definindo os dados para analisar a quantidade de ganhadores por ano
x_ganhadores = []
y_ganhadores = []

for k, v in dict(ganhadores).items():
    x_ganhadores.append(k)
    y_ganhadores.append(v)

**Análise com o Plotly**

In [ ]:
grafico_linha_go(x_ganhadores, y_ganhadores, 'Ganhadores x Ano', 'Ano', 'Quantidade de Ganhadores')

In [ ]:
grafico_linha_go(x_concursos, y_concursos, 'Concursos x Ano', 'Ano', 'Quantidade de Concursos')

**Análise com o Matplotlib**

In [ ]:
grafico_linha_plt(x_ganhadores, y_ganhadores, 'Ganhadores x Ano', 'Ano', 'Quantidade de Ganhadores')

In [ ]:
grafico_linha_plt(x_concursos, y_concursos, 'Concursos x Ano', 'Ano', 'Quantidade de Concursos')

In [ ]:
# Definindo últimos anos
ano = str(base['Ano'].astype('int64').max() - 4)

In [ ]:
# Quantidade de concursos por mês nos últimos 5 anos
base.groupby(['Ano', 'Mes'])['Concurso'].count()[ano:]

In [ ]:
# Quantidade de ganhadores por mês nos últimos 5 anos
base.groupby(['Ano', 'Mes'])['Ganhou'].sum()[ano:]

In [ ]:
# Quantidade média de ganhadores por concurso em cada mês nos últimos 5 anos
base.groupby(['Ano', 'Mes'])['Ganhou'].mean()[ano:].round()

_A partir de 2012, a quantidade de concurso começou a manter um padrão linear. Com base neste fato, iremos análisar o intervalo de tempo a partir deste ano._

In [ ]:
# Perído de análise
ano_inicio = '2012'

In [ ]:
# Dados análise
analise = base[base['Ano'] >= ano_inicio]

In [ ]:
# Obtém os indices dos concursos que possuem e não possuem vencedores de acordo com o período estipulado
indice_c = base[(base['Ganhou'] > 0) & (base['Ano'] >= ano_inicio)]
indice_s = base[(base['Ganhou'] == 0) & (base['Ano'] >= ano_inicio)]

In [ ]:
# Quantidade de concursos com e sem vencedores
s_vencedor = len(indice_s)
c_vencedor = len(indice_c)

In [ ]:
# Quantidade
print(f'Qtde concurso sem vencedores: {s_vencedor} \nQtde concurso com vencedores: {c_vencedor}\nTotal: {len(analise)}')

## ANÁLISE DE SÉRIE TEMPORAL

In [ ]:
# Reajustando os dados
ts = analise.query('Ano <= "2020"')
ts.reset_index(drop=True, inplace=True)

In [ ]:
# Reajustando o campo de data
for indice, valor in ts['Data Sorteio'].items():
    ts.loc[indice, 'Data Sorteio'] = datetime.strptime(valor.strftime('%Y-%m'), '%Y-%m')

In [ ]:
# Agregando os dados
ts = ts.groupby('Data Sorteio')['Ganhou'].sum()

In [ ]:
# Quantidade de meses no perído que será avaliado
len(ts)

**Será avaliado no período:**

1 - _Tendência_  
2 - _sazionalidade_  
3 - _Variação_

In [ ]:
grafico_linha_go(ts.index, ts.values, 'Ganhadores x Período', 'Período', 'Ganhadores');

In [ ]:
# Decomposição da série temporal
decomposicao = seasonal_decompose(ts)
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
aleatorio = decomposicao.resid

In [ ]:
# Analisa a tendência
plt.plot(tendencia);

In [ ]:
# Analisa a sazionalidade
plt.plot(sazonal);

In [ ]:
# Analisa as variações no período
plt.plot(aleatorio);

In [ ]:
# Agrupado
plt.figure(figsize=(15, 8))

plt.subplot(4,1,1)
plt.plot(ts, label = 'Original', color='green', marker='o', linestyle='solid', linewidth=1.5, markersize=3)
plt.legend(loc = 'best')

plt.subplot(4,1,2)
plt.plot(tendencia, label = 'Tendência', color='cyan', marker='None', linestyle='solid', linewidth=1.5, markersize=3)
plt.legend(loc = 'best')

plt.subplot(4,1,3)
plt.plot(sazonal, label = 'Sazonalidade', color='brown', marker='o', linestyle='solid', linewidth=1.5, markersize=3)
plt.legend(loc = 'best')

plt.subplot(4,1,4)
plt.plot(aleatorio, label = 'Aletório', color='red', marker='o', linestyle='solid', linewidth=1.5, markersize=3)
plt.legend(loc = 'best')
plt.tight_layout()

**PREVISÃO DE GANHADORES PARA OS PRÓXIMOS CONCURSOS**  

In [ ]:
# Obtendo os parâmetros de ajuste do modelo
param = auto_arima(ts, m=12, seasonal=True, trace=False, suppress_warnings=True)
param.summary()

In [ ]:
# Criação e visualização das informações do modelo
modelo = ARIMA(ts, order=(0, 1, 1))
treino_modelo = modelo.fit()
treino_modelo.summary()

In [ ]:
# Previsões para os próximos meses (steps = quantidade de meses)
previsao = treino_modelo.forecast(steps=12)[0]

In [ ]:
previsao

In [ ]:
# Série original
plt.figure(figsize=(15, 4))
serie_original = ts.plot()

In [ ]:
# Plotando as previsões
treino_modelo.plot_predict('2020-06-01', '2021-01-01', ax=serie_original, plot_insample=True)